In [1]:
import pandas as pd
import random
from collections import defaultdict, Counter
from tqdm.notebook import tqdm
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import numpy as np
from sklearn.metrics import f1_score
import os
import torch
%matplotlib inline


In [18]:
all_data=pd.read_csv('WikiLarge_Train.csv')

In [19]:
all_data.head()
all_data.columns=['original_text','labels']

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(all_data, all_data['labels']):
    strat_train_set=all_data.iloc[train_index]
    strat_test_set=all_data.iloc[test_index]
strat_train_set['labels'].value_counts()/len(strat_train_set)

0    0.5
1    0.5
Name: labels, dtype: float64

In [5]:
cuda_available = torch.cuda.is_available()
cuda_available

True

In [31]:
model_args = ClassificationArgs(num_train_epochs=3,output_dir='670_final_project_output_classify_epoch_3/',save_steps=20000)

# Note here that we're saying to use CUDA, which has the model run on a GPU. If you don't have
# a GPU, set this to False.
model = ClassificationModel(
    'distilbert', 'distilbert-base-uncased', args=model_args, use_cuda=True
)


model.train_model(strat_train_set)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

(125031, 0.5007432502612649)

In [32]:
result_classify, model_outputs_classify, wrong_predictions_classify = model.eval_model(strat_test_set, f1=f1_score)

# The results contain all the built-in metrics, as well as our kwarg metric
print(result_classify)

/home/sqsq/.local/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:1028: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(



{'mcc': 0.4959910627454017, 'tp': 32307, 'tn': 30010, 'fp': 11667, 'fn': 9370, 'auroc': 0.829134555833215, 'auprc': 0.8206594306599773, 'f1': 0.7543869890602563, 'eval_loss': 0.5667427611862615}


In [21]:
testing=pd.read_csv('WikiLarge_Test.csv')

In [22]:
testing.head(200)

,id,original_text,label
0,0,-2011,NaN
1,1,-2011,NaN
2,2,-2000,NaN
3,3,-1997,NaN
4,4,1.636,NaN
...,...,...,...
195,195,'' Dont Panic '' -- 2:38,NaN
196,196,'' Dysphemism '' may be either offensive or me...,NaN
197,197,'' Dysphemism '' or '' malphemism '' may be ei...,NaN
198,198,'' Eleanor Rigby '' combines Paul McCartney 's...,NaN


In [23]:
all_test=list(testing['original_text'])

In [24]:
all_test

['-2011',
 '-2011',
 '-2000',
 '-1997',
 '1.636',
 '7.74',
 '8.557',
 '17.46',
 '21',
 '1945',
 '1947',
 '1996',
 '2003',
 '2004',
 '2004',
 '9/23/1930',
 '3-Jun-95',
 '1/1/2002',
 '14-Feb-02',
 '5-Jun-05',
 '20-Aug-07',
 '-',
 '-',
 '-',
 '-',
 '-',
 '--',
 '--',
 '--',
 '--',
 '--',
 "'' ''",
 "' - 2000",
 "' -- 2003 direct-to-video sequel to the 1973 film .",
 "-- '' Gregory IX had in fact excommunicated Frederick before he left Sicily the second time '' The Seventh , Eighth and Ninth Crusades resulted in Mamluk and Hafsid victories , as the Ninth Crusade marked the end of the Crusades in the Middle East .",
 "' , a TV series based on the famous video game character",
 "' , by Vangelis",
 "'' ... and no Warrants shall issue , but upon probable cause , supported by Oath or affirmation , and particularly describing the place to be searched , and the persons or things to be seized . ''",
 "'' ... This was represented to Varus as an occasion which required his prompt attendance at the s

In [37]:
predictions, raw_outputs = model.predict(all_test)

In [38]:
results=pd.DataFrame({'id':list(testing['id']), 'label':predictions})

In [39]:
results.to_csv('results.csv',index=None)

In [40]:
len(results)

119092